## Getting the code

In [1]:
PROGRAM=cmatrix
pull-lp-source $PROGRAM jammy

Found cmatrix 2.0-3 in jammy
[=====================================================>]100%
Public key not found, could not verify signature
[=====================================================>]100%
[=====================================================>]100%


In [2]:
ls

cmatrix-2.0                  cmatrix_2.0.orig.tar.gz  LICENSE
cmatrix_2.0-3.debian.tar.xz  do_homework.ipynb        README.md
cmatrix_2.0-3.dsc            getting_set_up.ipynb


## Adding `testing.sh` to source code

In [11]:
MY_SCRIPT="cmatrix-2.0/testing.sh"

cat > "$MY_SCRIPT" <<"EOF"
#!/bin/bash -

MESSAGE="this is a test from Clymber Loong"

if [ "$1" == "stdout" ] ; then
    echo "$MESSAGE" >&1
else
    echo "$MESSAGE" >&2
fi

exit $?
EOF

chmod +x "$MY_SCRIPT"

In [15]:
/bin/bash $MY_SCRIPT        # echo to stderr
/bin/bash $MY_SCRIPT stdout # echo to stdout

this is a test from Clymber Loong
this is a test from Clymber Loong


## Adding `testing.sh` to building and installation procedure

In [16]:
CMAKE_LISTS='cmatrix-2.0/CMakeLists.txt' # used by cmake
AUTOTOOLCFG='cmatrix-2.0/Makefile.am' # used by autoreconf

echo 'install(PROGRAMS testing.sh DESTINATION bin)' >> "$CMAKE_LISTS"
echo 'bin_SCRIPTS = testing.sh' >> "$AUTOTOOLCFG"

## Add package maintainer scripts

In [19]:
POSTINST='cmatrix-2.0/debian/postinst'

cat > "$POSTINST" <<"EOF"
#!/bin/bash -
set -e

if [ "$1" == "configure" ]; then
    testing.sh stdout
fi

exit $?
EOF

chmod +x "$POSTINST"